In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup as bs

import pandas as pd
import time



In [14]:
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 1000)

In [15]:
service = Service(executable_path=ChromeDriverManager().install())

In [70]:
def get_comments(service, df, webtoon_id, episode_no):
    TIMEOUT = 1   
    driver = webdriver.Chrome(service=service)
    
    url = 'https://comic.naver.com/bestChallenge/detail?'
    episode_url = url + 'titleId=' + str(webtoon_id) + '&no='+str(episode_no)
    print(episode_url)
    driver.get(episode_url) # 새창 열기
    driver.implicitly_wait(TIMEOUT)
    
    # 댓글창에 해당하는 프레임 선택
    driver.switch_to.frame('commentIframe')
    
    # 수집 편의를 위한 클린봇 댓글 필터링 기능 해제
    # (클린봇이 부적절한 표현을 감지한 댓글입니다.) 제거
    driver.find_element(By.CLASS_NAME,'u_cbox_cleanbot_setbutton').click()
    time.sleep(0.4)
    driver.find_element(By.CLASS_NAME,'u_cbox_layer_cleanbot2_checkbox').click()
    time.sleep(0.4)
    driver.find_element(By.CLASS_NAME,'u_cbox_layer_cleanbot2_extrabtn').click()
    time.sleep(0.4)

    # 전체 댓글 클릭
    driver.find_element(By.CSS_SELECTOR,'#cbox_module > div > div.u_cbox_view_comment > a').click()
    time.sleep(1)
    
    # "맨뒤" 클릭하여 전체 페이지 수 조회
    try:
        driver.find_element(By.CSS_SELECTOR,'#cbox_module > div > div.u_cbox_paginate > div > a.u_cbox_next.u_cbox_next_end > span.u_cbox_cnt_page').click()
        time.sleep(1)
        strong = driver.find_element(By.CSS_SELECTOR,'#cbox_module > div > div.u_cbox_paginate > div > strong')
        num_pages = strong.get_attribute('data-param')  # class_ = a.get_attribute('class')
        print("num_pages: ", num_pages)     
    
        # "맨앞" 으로 이동
        driver.find_element(By.CSS_SELECTOR,'#cbox_module > div > div.u_cbox_paginate > div > a.u_cbox_pre.u_cbox_pre_end > span.u_cbox_cnt_page').click()
        time.sleep(1)
    except: 
         num_pages=1
         print("num_pages: ", num_pages)     
            
    # 0 ~ 818
    for page in range(int(num_pages)):

        #최근 열린 탭으로 전환
        driver.switch_to.window(driver.window_handles[-1]) 
        driver.switch_to.frame("commentIframe")
        soup = bs(driver.page_source,"html.parser")

        # 날짜 추출
        dates = soup.findAll("span", {"class":["u_cbox_date"]})
        dates = [date.text for date in dates]        

        # 삭제된 댓글 제외하고 모든 댓글 추출
        reviews = soup.findAll("span", {"class":["u_cbox_contents"]})
        reviews = [review.get_text() for review in reviews]
        
   
        df_page = pd.DataFrame(data=[], columns=['Episode','Date','Review'])
        df_page['Date'] = dates
        df_page['Review'] = reviews
        df_page['Episode'] = df_page['Episode'].fillna('23') 
        
        df = pd.concat([df,df_page])      
        print(page+1, " page Done.")        

        if (page+1) == int(num_pages):
            return df
        
        # 다음 페이지로 이동
        a=driver.find_element(By.CSS_SELECTOR, "#cbox_module > div > div.u_cbox_paginate > div > strong + a").click()        
        a.send_keys('\n')
        time.sleep(0.4)
        
    driver.close()
    return df

df = pd.DataFrame(data=[], columns=['Episode','Date','Review'])
df = get_comments(service, df, 783797, 29) # 전지적 독자 시점 - 에피소드 1
df

df.to_csv("./best/23화.csv",index = False)

https://comic.naver.com/bestChallenge/detail?titleId=783797&no=29
num_pages:  1
1  page Done.


In [71]:
df

,Episode,Date,Review
0,23,4일 전,셀트 응원하다가 은하 생각나서 정주행 다시 하는데 은하야 미안하다은하도 너무 좋은 ...
1,23,4일 전,니 여친라는 말에 태클거는 거 너무 웃겨요ㅠㅠㅠ세현아 미안하다 나도 셀트 응원해ㅠㅠ...
2,23,6일 전,허걱 신캐 나와서 설렜는데 냅다 세현이 멱살을 잡아버리다니 ㄷㄷㄷ 세현이 과거 떡밥...
3,23,6일 전,클로지오 시절에 대체 뭘했길래 저렇게 싫어하지... 세현이만 좀 불쌍... 기억 없...
4,23,6일 전,방금 물리 사제를 본 것 같은데...주먹이..
5,23,6일 전,작가님..연재주기 짧은게 무슨 상관입니까!!!이렇게 재미있게 써주시는데!!! 이대로...
6,23,6일 전,작가님 이번화도 ㄹㅈㄷ 찍네요!!


In [72]:
import pandas as pd
from glob import glob

file_names = glob("C:/python/best/*.csv") #폴더 내의 모든 csv파일 목록을 불러온다
total = pd.DataFrame() #빈 데이터프레임 하나를 생성한다

for file_name in file_names:
    temp = pd.read_csv(file_name, sep='구분자', encoding='utf-8') #csv파일을 하나씩 열어 임시 데이터프레임으로 생성한다
    total = pd.concat([total, temp]) #전체 데이터프레임에 추가하여 넣는다

total.to_csv("C:/python/best/bestwebtoon_total.csv")


C:\Users\nirae\AppData\Local\Temp\ipykernel_33072\477539580.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  temp = pd.read_csv(file_name, sep='구분자', encoding='utf-8') #csv파일을 하나씩 열어 임시 데이터프레임으로 생성한다
C:\Users\nirae\AppData\Local\Temp\ipykernel_33072\477539580.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  temp = pd.read_csv(file_name, sep='구분자', encoding='utf-8') #csv파일을 하나씩 열어 임시 데이터프레임으로 생성한다
C:\Users\nirae\AppData\Local\Temp\ipykernel_33072\477539580.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and diffe

C:\Users\nirae\AppData\Local\Temp\ipykernel_33072\477539580.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  temp = pd.read_csv(file_name, sep='구분자', encoding='utf-8') #csv파일을 하나씩 열어 임시 데이터프레임으로 생성한다
C:\Users\nirae\AppData\Local\Temp\ipykernel_33072\477539580.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  temp = pd.read_csv(file_name, sep='구분자', encoding='utf-8') #csv파일을 하나씩 열어 임시 데이터프레임으로 생성한다
